In [5]:
from nilearn import image

from nipype.pipeline.engine import Node, Workflow
from nipype.interfaces import fsl
import nipype.interfaces.utility as niu
import nipype.interfaces.ants as ants  

In [24]:
from  pathlib import Path
bids_folder = Path('/data/ds-retsupp')
subject, session = '01', 1

workflow = Workflow(name=f'bet_workflow_sub-{subject}_session-{session}', base_dir='/tmp/workflow_folders')
input_node = Node(niu.IdentityInterface(fields=['t1w_file', 'inv2_file']),
                  name='input_node')

input_node.inputs.t1w_file = bids_folder / f'sub-{subject}/ses-{session}/anat/sub-{subject}_ses-{session}_T1w.nii.gz'
input_node.inputs.inv2_file = bids_folder / f'sub-{subject}/ses-{session}/anat/sub-{subject}_ses-{session}_inv-2_MP2RAGE.nii.gz'

bet = Node(fsl.BET(mask=True),
            name='bet')

def create_t1w(t1w_file, inv2_file, mask_file):
    import os
    from nilearn import image
    from nipype.utils.filemanip import filename_to_list, split_filename

    
    # Load the T1w and inv2 images
    t1w_img = image.load_img(t1w_file)
    inv2_img = image.load_img(inv2_file)
    mask_img = image.load_img(mask_file)

    t1w_processed = image.math_img('np.where(mask, t1w, (inv2/inv2.max())*t1w)', 
                                      t1w=t1w_img,
                                      inv2=inv2_img,
                                      mask=mask_img)

    # Save the processed T1w image
    # Split the original filename into parts
    _, base, ext = split_filename(t1w_file)

    # Create the new filename with a suffix
    output_file = os.path.abspath(f"{base}_processed{ext}")
    t1w_processed.to_filename(output_file)
    return output_file

cleaner = Node(niu.Function(input_names=['t1w_file', 'inv2_file', 'mask_file'],
                    output_names=['t1w_processed'],
                    function=create_t1w),
                    name='cleaner')

n4_biasfield = Node(ants.N4BiasFieldCorrection(),
                    name='n4_biasfield')

workflow.connect(input_node, 'inv2_file', bet, 'in_file')
# workflow.connect(input_node, 't1w_file', masker, 'in_file')
# workflow.connect(bet, 'mask_file', n4_biasfield, 'mask_image')
# workflow.connect(input_node, 't1w_file', n4_biasfield, 'input_image')
# workflow.connect(n4_biasfield, 'output_image', cleaner, 't1w_file')
workflow.connect(input_node, 'inv2_file', cleaner, 'inv2_file')
workflow.connect(input_node, 't1w_file', cleaner, 't1w_file')
workflow.connect(bet, 'mask_file', cleaner, 'mask_file')


workflow.run()

250512-08:21:18,141 nipype.workflow INFO:
	 Workflow bet_workflow_sub-01_session-1 settings: ['check', 'execution', 'logging', 'monitoring']
250512-08:21:18,154 nipype.workflow INFO:
	 Running serially.
250512-08:21:18,155 nipype.workflow INFO:
	 [Node] Setting-up "bet_workflow_sub-01_session-1.bet" in "/private/tmp/workflow_folders/bet_workflow_sub-01_session-1/bet".
250512-08:21:18,159 nipype.workflow INFO:
	 [Node] Cached "bet_workflow_sub-01_session-1.bet" - collecting precomputed outputs
250512-08:21:18,159 nipype.workflow INFO:
	 [Node] "bet_workflow_sub-01_session-1.bet" found cached.
250512-08:21:18,160 nipype.workflow INFO:
	 [Node] Setting-up "bet_workflow_sub-01_session-1.cleaner" in "/private/tmp/workflow_folders/bet_workflow_sub-01_session-1/cleaner".
250512-08:21:18,162 nipype.workflow INFO:
	 [Node] Outdated cache found for "bet_workflow_sub-01_session-1.cleaner".
250512-08:21:18,215 nipype.workflow INFO:
	 [Node] Executing "cleaner" <nipype.interfaces.utility.wrappers.F

In [32]:
from  pathlib import Path
bids_folder = Path('/data/ds-retsupp')
subject, session = '01', 1

workflow = Workflow(name=f'bet_workflow_sub-{subject}_session-{session}', base_dir='/tmp/workflow_folders')
input_node = Node(niu.IdentityInterface(fields=['t1w_file', 'inv2_file']),
                  name='input_node')

input_node.inputs.t1w_file = bids_folder / f'sub-{subject}/ses-{session}/anat/sub-{subject}_ses-{session}_T1w.nii.gz'
input_node.inputs.inv2_file = bids_folder / f'sub-{subject}/ses-{session}/anat/sub-{subject}_ses-{session}_inv-2_MP2RAGE.nii.gz'

bet = Node(fsl.BET(mask=True),
            name='bet')

def create_t1w(inv2_file, t1w_file):
    import os
    from nilearn import image
    from nipype.utils.filemanip import filename_to_list, split_filename

    
    # Load the T1w and inv2 images
    t1w_img = image.load_img(t1w_file)
    inv2_img = image.load_img(inv2_file)

    t1w_processed = image.math_img('inv2/inv2.max()*t1w', 
                                      t1w=t1w_img,
                                      inv2=inv2_img)

    # Save the processed T1w image
    # Split the original filename into parts
    _, base, ext = split_filename(t1w_file)

    # Create the new filename with a suffix
    output_file = os.path.abspath(f"{base}_processed{ext}")
    t1w_processed.to_filename(output_file)
    return output_file

cleaner = Node(niu.Function(input_names=['t1w_file', 'inv2_file'],
                    output_names=['t1w_processed'],
                    function=create_t1w),
                    name='cleaner')


n4_biasfield = Node(ants.N4BiasFieldCorrection(),
                    name='n4_biasfield')

workflow.connect(input_node, 'inv2_file', bet, 'in_file')
workflow.connect(bet, 'mask_file', n4_biasfield, 'mask_image')
workflow.connect(cleaner, 't1w_processed', n4_biasfield, 'input_image')
workflow.connect(input_node, 'inv2_file', cleaner, 'inv2_file')
workflow.connect(input_node, 't1w_file', cleaner, 't1w_file')

workflow.run()

250512-08:29:53,166 nipype.workflow INFO:
	 Workflow bet_workflow_sub-01_session-1 settings: ['check', 'execution', 'logging', 'monitoring']
250512-08:29:53,172 nipype.workflow INFO:
	 Running serially.
250512-08:29:53,172 nipype.workflow INFO:
	 [Node] Setting-up "bet_workflow_sub-01_session-1.bet" in "/private/tmp/workflow_folders/bet_workflow_sub-01_session-1/bet".
250512-08:29:53,175 nipype.workflow INFO:
	 [Node] Cached "bet_workflow_sub-01_session-1.bet" - collecting precomputed outputs
250512-08:29:53,175 nipype.workflow INFO:
	 [Node] "bet_workflow_sub-01_session-1.bet" found cached.
250512-08:29:53,176 nipype.workflow INFO:
	 [Node] Setting-up "bet_workflow_sub-01_session-1.cleaner" in "/private/tmp/workflow_folders/bet_workflow_sub-01_session-1/cleaner".
250512-08:29:53,210 nipype.workflow INFO:
	 [Node] Executing "cleaner" <nipype.interfaces.utility.wrappers.Function>
250512-08:29:55,467 nipype.workflow INFO:
	 [Node] Finished "cleaner", elapsed time 2.255287s.
250512-08:29:

In [25]:

multiplier = Node(fsl.ImageMaths(op_string='-mul'),
                  name='multiplier')


In [26]:
multiplier.inputs


args = <undefined>
environ = {'FSLOUTPUTTYPE': 'NIFTI_GZ'}
in_file = <undefined>
in_file2 = <undefined>
mask_file = <undefined>
op_string = -mul
out_data_type = <undefined>
out_file = <undefined>
output_type = NIFTI_GZ
suffix = <undefined>

In [20]:
!ls /data/ds-retsupp/sub-01/ses-1/anat/

sub-01_ses-1_-inv-1_MP2RAGE.nii.gz sub-01_ses-1_T1w.nii.gz
sub-01_ses-1_-inv-2_MP2RAGE.nii.gz sub-01_ses-1_T2w.nii.gz


In [16]:
n4_biasfield.inputs


args = <undefined>
bias_image = <undefined>
bspline_fitting_distance = <undefined>
bspline_order = <undefined>
convergence_threshold = <undefined>
copy_header = False
dimension = 3
environ = {'NSLOTS': '1'}
histogram_sharpening = <undefined>
input_image = <undefined>
mask_image = <undefined>
n_iterations = <undefined>
num_threads = 1
output_image = <undefined>
rescale_intensities = False
save_bias = False
shrink_factor = <undefined>
weight_image = <undefined>

In [11]:
masker.outputs


out_file = <undefined>